In [3]:
from tg.grammar_ru.components.plain_context_builder import PlainContextBuilder
from tg.common.ml.batched_training import IndexedDataBundle
from sklearn.metrics import confusion_matrix
import math
import torch
from typing import List, Union
from tg.grammar_ru.common import Separator
import plotly.express as px
from yo_fluq_ds import Queryable, Query, fluq
from tg.grammar_ru.components.yandex_delivery.training_logs import S3TrainingLogsLoader, TrainingLogsViewer
from tg.grammar_ru.components.yandex_storage.s3_yandex_helpers import S3YandexHandler
import numpy as np
import pandas as pd
from tg.grammar_ru.common import Loc
from collections import defaultdict

import re

from tg.common import DataBundle
from tg.common.ml.batched_training import train_display_test_split
from tg.grammar_ru.features import PyMorphyFeaturizer

from tg.grammar_ru.corpus import ITransfuseSelector
from nltk.stem import SnowballStemmer
from tg.projects.agreement.bundles_tools import _print_thrown
import ast
import seaborn as sns
from plotly.offline import download_plotlyjs, init_notebook_mode, plot
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from tg.grammar_ru.common import Loc
from tg.grammar_ru.corpus import CorpusReader, CorpusBuilder, BucketCorpusBalancer
from tg.grammar_ru.corpus.corpus_reader import read_data
import os
from pathlib import Path
from dotenv import load_dotenv
load_dotenv(Loc.root_path / 'environment.env')


pd.set_option('display.max_rows', 500)


In [4]:
# # second_declension_ends = set("а я ы и е у ю ой ёй ей ою ".split())

# class NounAgreementTrainIndexBuilder(ITransfuseSelector):
#     first_declension_ends = set("а я ы и е у ю ой ёй ей ".split())
#     # дядей, землёй Note в печатных текстах, наверное, ё заменяют на е

#     POSSIBLE_ENDINGS = first_declension_ends
#     ends_list = sorted(list(first_declension_ends))
#     num_by_end = {e: n for n, e in enumerate(ends_list)}

#     def _extract_ending(self, word: str):
#         for possible_ending in self.POSSIBLE_ENDINGS:
#             if word.lower().endswith(possible_ending):
#                 return possible_ending
#         return np.nan

#     def __init__(self):
#         self.pmf = PyMorphyFeaturizer()
#         # self.snowball = SnowballStemmer(language="russian")
#         self.norm_endings_nums = {e: i for i,
#                                   e in enumerate(['я', 'а'])}
#         # self.endings_nums = {e: i for i, e in enumerate(ALL_ENDS_list)}

#     def _extract_norm_ending(self, word_in_norm_form: str):
#         for possible_ending in self.norm_endings_nums.keys():
#             if word_in_norm_form.lower().endswith(possible_ending):
#                 return possible_ending
#         return np.nan

#     def select(self, source, df, toc_row):
#         db = DataBundle(src=df)
#         self.pmf.featurize(db)
#         morphed = db.data_frames['pymorphy']
#         morphed.replace({np.nan: 'nan'}, inplace=True)
#         nouns = df[(morphed.POS == "NOUN")].copy()  # TODO delete
#         # return morphed[(morphed.POS == "NOUN")]
#         df['is_target'] = False
#         df['declension_type'] = -1

#         nouns['ending'] = (nouns.word
#                            .apply(self._extract_ending))

#         morphed_nouns = morphed.loc[nouns.index]
#         nouns['norm_ending'] = (morphed_nouns.normal_form
#                                 .apply(self._extract_norm_ending))

#         undefined_ending_mask = (nouns.norm_ending.isnull() |
#                                  nouns.ending.isnull())
#         thrown = list(set(nouns[undefined_ending_mask].word))

#         nouns = nouns[~undefined_ending_mask]
#         nouns['declension_type'] = 1
#         # adjectives.norm_ending.replace(            self.norm_endings_nums)

#         nouns['label'] = nouns.ending.map(self.num_by_end)
#         thrown.extend(nouns[nouns.label.isnull()].word)
#         nouns = nouns[~nouns.label.isnull()]

#         df.loc[nouns.index, 'declension_type'] = nouns['declension_type']
#         df.declension_type = df.declension_type.astype(int)
#         df['label'] = -1
#         df.loc[nouns.index, 'label'] = nouns.label
#         df.loc[nouns.index, 'is_target'] = True
#         _print_thrown(thrown, Loc.temp_path / "noun_undefined_ending.txt")
#         return [df]

#     @staticmethod
#     def build_index_from_src(src_df):
#         df = src_df.loc[src_df.is_target][[
#             'word_id', 'sentence_id', 'declension_type', 'label']].copy()
#         df = df.reset_index(drop=True)
#         df.index.name = 'sample_id'
#         df['split'] = train_display_test_split(df)
#         return df


In [5]:
from tg.grammar_ru.corpus.corpus_reader import CorpusReader
from tg.projects.agreement.bundle import NounAgreementTrainIndexBuilder
selector = NounAgreementTrainIndexBuilder()
source = Loc.corpus_path / 'prepare/balanced/books&pub_60K_balanced_feat.zip'
reader = CorpusReader(source)
toc = reader.get_toc()
frame_index_in_corpus = 0
frame = reader.get_frames().first()
toc_row = toc.iloc[frame_index_in_corpus].to_dict()
frame_index_in_corpus += 1
dfs = selector.select(source, frame, toc_row)
            


In [7]:
df = dfs[0]
df[~df.norm_ending.isnull()]

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,file_id,corpus_id,original_corpus_id,original_word_id,original_sentence_id,original_paragraph_id,updated,is_target,declension_type,norm_ending,label
18,18,0,18,0,0,книжки,ru,6,45f6d5b5-3e14-4346-b848-003eed141143,books&pub_60K_balanced_feat.zip,books.base.zip,92,4,1,False,True,1,1.0,3
48,48,6,4,6,0,орхидей,ru,7,45f6d5b5-3e14-4346-b848-003eed141143,books&pub_60K_balanced_feat.zip,books.base.zip,510,63,55,False,True,1,0.0,2
79,79,77,0,76,1,Строчки,ru,7,45f6d5b5-3e14-4346-b848-003eed141143,books&pub_60K_balanced_feat.zip,books.base.zip,10728,10713,10711,False,True,1,1.0,3
111,111,78,23,76,0,неделе,ru,6,45f6d5b5-3e14-4346-b848-003eed141143,books&pub_60K_balanced_feat.zip,books.base.zip,10769,10715,10711,False,True,1,0.0,1
118,118,78,30,76,1,голове,ru,6,45f6d5b5-3e14-4346-b848-003eed141143,books&pub_60K_balanced_feat.zip,books.base.zip,10776,10715,10711,False,True,1,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50117,50117,49521,25,49516,0,школе,ru,5,45f6d5b5-3e14-4346-b848-003eed141143,books&pub_60K_balanced_feat.zip,books.base.zip,445460,443253,443167,False,True,1,1.0,1
50163,50163,49522,22,49517,0,руки,ru,4,45f6d5b5-3e14-4346-b848-003eed141143,books&pub_60K_balanced_feat.zip,books.base.zip,445673,443264,443170,False,True,1,1.0,3
50166,50166,49522,25,49517,0,медовуху,ru,8,45f6d5b5-3e14-4346-b848-003eed141143,books&pub_60K_balanced_feat.zip,books.base.zip,445676,443264,443170,False,True,1,1.0,5
50225,50225,49525,10,49519,0,Выручай-комнате,ru,15,45f6d5b5-3e14-4346-b848-003eed141143,books&pub_60K_balanced_feat.zip,books.base.zip,445913,443280,443180,False,True,1,1.0,1


In [6]:
dfs[0].is_target.sum()

2414

In [7]:
len(dfs[0])

49971

In [4]:
from tg.common import DataBundle


db = DataBundle.load(Loc.data_cache_path / \
    f'bundles/agreement/noun_norm_e_mid50')


In [10]:
db.src[db.src.is_target]

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,file_id,corpus_id,original_corpus_id,original_word_id,original_sentence_id,original_paragraph_id,updated,is_target,declension_type,norm_ending,label
18,18,0,18,0,0,книжки,ru,6,453e2ab7-cfbc-4cbc-8d28-7b401c8a045e,raw.zip,books.base.zip,18,0,0,False,True,1,1.0,3
48,48,6,4,6,0,орхидей,ru,7,453e2ab7-cfbc-4cbc-8d28-7b401c8a045e,raw.zip,books.base.zip,48,6,6,False,True,1,0.0,2
68,68,10,0,9,1,Строчки,ru,7,453e2ab7-cfbc-4cbc-8d28-7b401c8a045e,raw.zip,books.base.zip,79,77,76,False,True,1,1.0,3
100,100,11,23,9,0,неделе,ru,6,453e2ab7-cfbc-4cbc-8d28-7b401c8a045e,raw.zip,books.base.zip,111,78,76,False,True,1,0.0,1
107,107,11,30,9,1,голове,ru,6,453e2ab7-cfbc-4cbc-8d28-7b401c8a045e,raw.zip,books.base.zip,118,78,76,False,True,1,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2499821,2499821,2452328,0,2451380,1,Трубку,ru,6,b0c53550-1eca-4468-bcff-7a51f7949ef0,raw.zip,books.base.zip,2525044,2522029,2521957,False,True,1,1.0,5
2499828,2499828,2452329,2,2451381,1,Тришей,ru,6,b0c53550-1eca-4468-bcff-7a51f7949ef0,raw.zip,books.base.zip,2525051,2522030,2521958,False,True,1,1.0,2
2499842,2499842,2452330,6,2451382,1,волей,ru,5,b0c53550-1eca-4468-bcff-7a51f7949ef0,raw.zip,books.base.zip,2525065,2522031,2521959,False,True,1,0.0,2
2499859,2499859,2452330,23,2451382,0,шутка,ru,5,b0c53550-1eca-4468-bcff-7a51f7949ef0,raw.zip,books.base.zip,2525082,2522031,2521959,False,True,1,1.0,0
